No.of total traces=1000

In [1]:
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier

# Toggle numba for speed (optional)
USE_NUMBA = False

KYBER_Q = 3329
QINV = -3327

def int16(x: int) -> int:
    x = x & 0xFFFF
    return x - 0x10000 if x & 0x8000 else x

def uint16(x: int) -> int:
    return x & 0xFFFF

def montgomery_reduce(a: int) -> int:
    t = int16(a)
    t = int16(t * QINV)
    res = (a - (int(t) * KYBER_Q)) >> 16
    return int16(res)

def fqmul(a: int, b: int) -> int:
    prod = int16(a) * int16(b)
    return uint16(montgomery_reduce(prod))

# Optional Numba implementation
if USE_NUMBA:
    try:
        from numba import njit

        @njit
        def fqmul_numba(a: int, b: int) -> int:
            ax = a & 0xFFFF
            if ax & 0x8000:
                ax -= 0x10000
            bx = b & 0xFFFF
            if bx & 0x8000:
                bx -= 0x10000
            prod = ax * bx
            res = (prod - (prod & 0xFFFF) * QINV * KYBER_Q) >> 16
            res &= 0xFFFF
            if res & 0x8000:
                res -= 0x10000
            return res & 0xFFFF

        fqmul_impl = fqmul_numba
    except Exception:
        fqmul_impl = fqmul
else:
    fqmul_impl = fqmul

# Hamming Weight Lookup
HW_LOOKUP = np.array([bin(i).count("1") for i in range(256)], dtype=np.uint8)

def load_traces(path):
    traces = np.load(path)
    return traces.astype(np.float32)

def load_nonces(path):
    nonces = np.load(path)
    if nonces.dtype == np.uint8:
        if nonces.shape[1] % 2 != 0:
            raise ValueError("Nonce byte array must have even number of columns")
        nonces16 = nonces[:, ::2].astype(np.uint16) + (nonces[:, 1::2].astype(np.uint16) << 8)
        return nonces16
    return nonces.astype(np.uint16)

def compress(raw: np.ndarray, cf: int) -> np.ndarray:
    N, L = raw.shape
    CL = (L + cf - 1) // cf
    out = np.zeros((N, CL), dtype=np.float32)
    for i in range(CL):
        start, end = i * cf, min(L, (i + 1) * cf)
        out[:, i] = raw[:, start:end].mean(axis=1)
    return out

def compute_between_class_poi_score(traces: np.ndarray, labels: np.ndarray, prof: int):
    Xp = traces[:prof]
    labs = labels[:prof]
    mu_all = Xp.mean(axis=0)
    score = np.zeros(Xp.shape[1], dtype=np.float64)
    classes = np.unique(labs)
    for c in classes:
        mask = labs == c
        if np.sum(mask) == 0:
            continue
        mu_c = Xp[mask].mean(axis=0)
        diff = (mu_c - mu_all) ** 2
        score += diff
    return score

# ---------------------------------------------------------
# NEW ATTACK FUNCTION WITH TOP-10 OUTPUT
# ---------------------------------------------------------
def attack_rf_hw_multiclass_top10(traces, nonces, prof, NUM_POI, which,
                                  chunk=2048, rf_n_estimators=200,
                                  rf_n_jobs=-1, random_state=0):

    N = traces.shape[0]
    att = N - prof
    if att <= 0:
        raise ValueError("Number of attack traces is zero or negative.")

    # ---------------------
    # Build HW labels
    # ---------------------
    idx = 2 if which == 0 else 5
    labels = np.zeros(prof, dtype=np.int8)
    for i in range(prof):
        val = int(nonces[i, idx])
        msb = (val >> 8) & 0xFF
        labels[i] = int(HW_LOOKUP[msb])

    # POI via between-class variance
    score = compute_between_class_poi_score(traces, labels, prof)
    pois = np.argsort(-score)[:NUM_POI]

    X_train = traces[:prof, pois]
    scaler = StandardScaler()
    X_train_s = scaler.fit_transform(X_train)

    clf = RandomForestClassifier(n_estimators=rf_n_estimators,
                                 n_jobs=rf_n_jobs,
                                 random_state=random_state)
    clf.fit(X_train_s, labels)

    X_attack = traces[prof:, pois]
    X_attack_s = scaler.transform(X_attack)
    probs = np.clip(clf.predict_proba(X_attack_s), 1e-15, 1.0)
    logprobs = np.log(probs)

    nonce_b = nonces[prof:, 1].astype(np.int64)
    guesses = np.arange(65536, dtype=np.int64)

    all_loglikes = np.full(65536, -np.inf, dtype=np.float64)
    fqmul_local = fqmul_impl

    classes_present = clf.classes_
    class_to_col = {int(c): i for i, c in enumerate(classes_present)}

    # ---------------------
    # CHUNK SWEEP
    # ---------------------
    for start in range(0, 65536, chunk):
        end = min(start + chunk, 65536)
        g_chunk = guesses[start:end]
        cols = g_chunk.shape[0]

        prod_chunk = np.empty((att, cols), dtype=np.uint16)
        for j in range(att):
            b = int(nonce_b[j])
            row = np.empty(cols, dtype=np.uint16)
            for k in range(cols):
                row[k] = fqmul_local(int(g_chunk[k]), b)
            prod_chunk[j] = row

        msb_chunk = ((prod_chunk >> 8) & 0xFF).astype(np.uint8)
        hw_chunk = HW_LOOKUP[msb_chunk]

        class_chunk = np.full(hw_chunk.shape, -1, dtype=np.int64)
        for hw_val in np.unique(hw_chunk):
            mask = (hw_chunk == hw_val)
            if hw_val in class_to_col:
                class_chunk[mask] = class_to_col[hw_val]
            else:
                # fallback to nearest class
                seen = np.array(classes_present)
                nearest = seen[np.argmin(np.abs(seen - hw_val))]
                class_chunk[mask] = class_to_col[int(nearest)]

        tot_loglike = np.zeros(cols, dtype=np.float64)
        for j in range(att):
            tot_loglike += logprobs[j, class_chunk[j]]

        all_loglikes[start:end] = tot_loglike

    # ---------------------
    # RESULTS + TOP-10
    # ---------------------
    best_guess = int(np.argmax(all_loglikes))
    best_ll = all_loglikes[best_guess]
    tied = np.where(all_loglikes == best_ll)[0].tolist()

    top10_idx = np.argsort(-all_loglikes)[:10]
    top10_scores = all_loglikes[top10_idx]
    top10_modq = [int(k) % KYBER_Q for k in top10_idx]

    print(f"\n========== TOP 10 KEY GUESSES for a{which} ==========")
    for i, (k, s, modq) in enumerate(zip(top10_idx, top10_scores, top10_modq), 1):
        print(f"{i:2d}. key={k:5d}   loglike={s:.4f}   mod_q={modq}")

    return best_guess, tied, top10_idx, top10_scores, top10_modq


poi=10

In [2]:


# ---------------------------------------------------------
# MAIN RUNNER
# ---------------------------------------------------------
if __name__ == "__main__":
    traces_file = "tracesA0.npy"
    nonces_file = "noncesA0.npy"
    compressF = 10
    NUM_POI = 10
    profFrac = 0.8
    CHUNK = 2048

    raw = load_traces(traces_file)
    nonces = load_nonces(nonces_file)
    N = 1000
    raw, nonces = raw[:N], nonces[:N]

    traces = compress(raw, compressF)
    prof = max(1, int(profFrac * N))

    
    rec0, ties0, top10_0, scores0, mod0 = \
        attack_rf_hw_multiclass_top10(traces, nonces, prof, NUM_POI, which=0)
    
    rec1, ties1, top10_1, scores1, mod1 = \
        attack_rf_hw_multiclass_top10(traces, nonces, prof, NUM_POI, which=1)

    true0 = int(nonces[0, 0])
    true1 = int(nonces[0, 3])

    print(f"\nRecovered a0={rec0} (true={true0})")
    print(f"Recovered a1={rec1} (true={true1})")
    print(f"Correct0? {rec0 == true0}")
    print(f"Correct1? {rec1 == true1}")



========== TOP 10 KEY GUESSES for a0 ==========
 1. key=  558   loglike=-174.3059   mod_q=558
 2. key= 3887   loglike=-241.9846   mod_q=558
 3. key=62765   loglike=-242.0320   mod_q=2843
 4. key= 7216   loglike=-309.3280   mod_q=558
 5. key=59436   loglike=-342.6053   mod_q=2843
 6. key=56107   loglike=-409.3160   mod_q=2843
 7. key=10545   loglike=-445.1981   mod_q=558
 8. key=52778   loglike=-477.2381   mod_q=2843
 9. key=13874   loglike=-512.8749   mod_q=558
10. key=49449   loglike=-543.8732   mod_q=2843

========== TOP 10 KEY GUESSES for a1 ==========
 1. key=   17   loglike=-191.0561   mod_q=17
 2. key=62224   loglike=-257.9797   mod_q=2302
 3. key= 3346   loglike=-258.2534   mod_q=17
 4. key=58895   loglike=-324.4150   mod_q=2302
 5. key=55566   loglike=-392.9420   mod_q=2302
 6. key= 6675   loglike=-426.1180   mod_q=17
 7. key=52237   loglike=-461.2934   mod_q=2302
 8. key=10004   loglike=-494.2963   mod_q=17
 9. key=13333   loglike=-561.9085   mod_q=17
10. key=16662   loglike=

poi=50

In [3]:


# ---------------------------------------------------------
# MAIN RUNNER
# ---------------------------------------------------------
if __name__ == "__main__":
    traces_file = "tracesA0.npy"
    nonces_file = "noncesA0.npy"
    compressF = 10
    NUM_POI = 50
    profFrac = 0.8
    CHUNK = 2048

    raw = load_traces(traces_file)
    nonces = load_nonces(nonces_file)
    N = 1000
    raw, nonces = raw[:N], nonces[:N]

    traces = compress(raw, compressF)
    prof = max(1, int(profFrac * N))

    rec0, ties0, top10_0, scores0, mod0 = \
        attack_rf_hw_multiclass_top10(traces, nonces, prof, NUM_POI, which=0)
    
    rec1, ties1, top10_1, scores1, mod1 = \
        attack_rf_hw_multiclass_top10(traces, nonces, prof, NUM_POI, which=1)

    true0 = int(nonces[0, 0])
    true1 = int(nonces[0, 3])

    print(f"\nRecovered a0={rec0} (true={true0})")
    print(f"Recovered a1={rec1} (true={true1})")
    print(f"Correct0? {rec0 == true0}")
    print(f"Correct1? {rec1 == true1}")



========== TOP 10 KEY GUESSES for a0 ==========
 1. key=  558   loglike=-155.0871   mod_q=558
 2. key=62765   loglike=-222.8266   mod_q=2843
 3. key= 3887   loglike=-223.7712   mod_q=558
 4. key= 7216   loglike=-290.1192   mod_q=558
 5. key=59436   loglike=-324.4631   mod_q=2843
 6. key=56107   loglike=-391.2184   mod_q=2843
 7. key=10545   loglike=-425.2725   mod_q=558
 8. key=52778   loglike=-459.1183   mod_q=2843
 9. key=13874   loglike=-492.8899   mod_q=558
10. key=17203   loglike=-526.4870   mod_q=558

========== TOP 10 KEY GUESSES for a1 ==========
 1. key=   17   loglike=-180.4490   mod_q=17
 2. key= 3346   loglike=-216.8787   mod_q=17
 3. key=62224   loglike=-247.8918   mod_q=2302
 4. key=58895   loglike=-314.5284   mod_q=2302
 5. key=55566   loglike=-383.1320   mod_q=2302
 6. key= 6675   loglike=-385.1762   mod_q=17
 7. key=52237   loglike=-451.6605   mod_q=2302
 8. key=10004   loglike=-453.3205   mod_q=17
 9. key=13333   loglike=-521.2151   mod_q=17
10. key=16662   loglike=-

poi=100

In [4]:


# ---------------------------------------------------------
# MAIN RUNNER
# ---------------------------------------------------------
if __name__ == "__main__":
    traces_file = "tracesA0.npy"
    nonces_file = "noncesA0.npy"
    compressF = 10
    NUM_POI = 100
    profFrac = 0.8
    CHUNK = 2048

    raw = load_traces(traces_file)
    nonces = load_nonces(nonces_file)
    N = 1000
    raw, nonces = raw[:N], nonces[:N]

    traces = compress(raw, compressF)
    prof = max(1, int(profFrac * N))

    
    rec0, ties0, top10_0, scores0, mod0 = \
        attack_rf_hw_multiclass_top10(traces, nonces, prof, NUM_POI, which=0)
    
    rec1, ties1, top10_1, scores1, mod1 = \
        attack_rf_hw_multiclass_top10(traces, nonces, prof, NUM_POI, which=1)

    true0 = int(nonces[0, 0])
    true1 = int(nonces[0, 3])

    print(f"\nRecovered a0={rec0} (true={true0})")
    print(f"Recovered a1={rec1} (true={true1})")
    print(f"Correct0? {rec0 == true0}")
    print(f"Correct1? {rec1 == true1}")



========== TOP 10 KEY GUESSES for a0 ==========
 1. key=  558   loglike=-151.4660   mod_q=558
 2. key=62765   loglike=-219.3721   mod_q=2843
 3. key= 3887   loglike=-220.2290   mod_q=558
 4. key= 7216   loglike=-286.2447   mod_q=558
 5. key=59436   loglike=-321.3828   mod_q=2843
 6. key=56107   loglike=-388.7243   mod_q=2843
 7. key=10545   loglike=-421.8975   mod_q=558
 8. key=52778   loglike=-456.7044   mod_q=2843
 9. key=13874   loglike=-489.4356   mod_q=558
10. key=17203   loglike=-523.1069   mod_q=558

========== TOP 10 KEY GUESSES for a1 ==========
 1. key=   17   loglike=-174.4357   mod_q=17
 2. key= 3346   loglike=-210.0746   mod_q=17
 3. key=62224   loglike=-242.1994   mod_q=2302
 4. key=58895   loglike=-309.8398   mod_q=2302
 5. key=55566   loglike=-378.4010   mod_q=2302
 6. key= 6675   loglike=-380.0586   mod_q=17
 7. key=10004   loglike=-417.1436   mod_q=17
 8. key=52237   loglike=-446.7046   mod_q=2302
 9. key=13333   loglike=-484.9412   mod_q=17
10. key=16662   loglike=-

poi=200

In [5]:


# ---------------------------------------------------------
# MAIN RUNNER
# ---------------------------------------------------------
if __name__ == "__main__":
    traces_file = "tracesA0.npy"
    nonces_file = "noncesA0.npy"
    compressF = 10
    NUM_POI = 200
    profFrac = 0.8
    CHUNK = 2048

    raw = load_traces(traces_file)
    nonces = load_nonces(nonces_file)
    N = 1000
    raw, nonces = raw[:N], nonces[:N]

    traces = compress(raw, compressF)
    prof = max(1, int(profFrac * N))


    rec0, ties0, top10_0, scores0, mod0 = \
        attack_rf_hw_multiclass_top10(traces, nonces, prof, NUM_POI, which=0)
   
    rec1, ties1, top10_1, scores1, mod1 = \
        attack_rf_hw_multiclass_top10(traces, nonces, prof, NUM_POI, which=1)

    true0 = int(nonces[0, 0])
    true1 = int(nonces[0, 3])

    print(f"\nRecovered a0={rec0} (true={true0})")
    print(f"Recovered a1={rec1} (true={true1})")
    print(f"Correct0? {rec0 == true0}")
    print(f"Correct1? {rec1 == true1}")



========== TOP 10 KEY GUESSES for a0 ==========
 1. key=  558   loglike=-153.1047   mod_q=558
 2. key=62765   loglike=-221.0199   mod_q=2843
 3. key= 3887   loglike=-221.5682   mod_q=558
 4. key= 7216   loglike=-287.5244   mod_q=558
 5. key=59436   loglike=-322.7492   mod_q=2843
 6. key=56107   loglike=-390.5059   mod_q=2843
 7. key=10545   loglike=-423.2735   mod_q=558
 8. key=52778   loglike=-458.4242   mod_q=2843
 9. key=13874   loglike=-490.9208   mod_q=558
10. key=17203   loglike=-524.5801   mod_q=558

========== TOP 10 KEY GUESSES for a1 ==========
 1. key=   17   loglike=-181.0522   mod_q=17
 2. key= 3346   loglike=-216.6137   mod_q=17
 3. key=62224   loglike=-248.8193   mod_q=2302
 4. key=58895   loglike=-316.1385   mod_q=2302
 5. key=55566   loglike=-384.6001   mod_q=2302
 6. key= 6675   loglike=-386.4654   mod_q=17
 7. key=10004   loglike=-423.4791   mod_q=17
 8. key=52237   loglike=-452.7894   mod_q=2302
 9. key=13333   loglike=-491.1058   mod_q=17
10. key=16662   loglike=-

poi=500

In [6]:


# ---------------------------------------------------------
# MAIN RUNNER
# ---------------------------------------------------------
if __name__ == "__main__":
    traces_file = "tracesA0.npy"
    nonces_file = "noncesA0.npy"
    compressF = 10
    NUM_POI = 500
    profFrac = 0.8
    CHUNK = 2048

    raw = load_traces(traces_file)
    nonces = load_nonces(nonces_file)
    N = 1000
    raw, nonces = raw[:N], nonces[:N]

    traces = compress(raw, compressF)
    prof = max(1, int(profFrac * N))


    rec0, ties0, top10_0, scores0, mod0 = \
        attack_rf_hw_multiclass_top10(traces, nonces, prof, NUM_POI, which=0)
    
    rec1, ties1, top10_1, scores1, mod1 = \
        attack_rf_hw_multiclass_top10(traces, nonces, prof, NUM_POI, which=1)

    true0 = int(nonces[0, 0])
    true1 = int(nonces[0, 3])

    print(f"\nRecovered a0={rec0} (true={true0})")
    print(f"Recovered a1={rec1} (true={true1})")
    print(f"Correct0? {rec0 == true0}")
    print(f"Correct1? {rec1 == true1}")



========== TOP 10 KEY GUESSES for a0 ==========
 1. key=  558   loglike=-165.1673   mod_q=558
 2. key=62765   loglike=-233.0671   mod_q=2843
 3. key= 3887   loglike=-233.2029   mod_q=558
 4. key= 7216   loglike=-270.2266   mod_q=558
 5. key=59436   loglike=-276.4690   mod_q=2843
 6. key=56107   loglike=-344.1458   mod_q=2843
 7. key=10545   loglike=-405.8591   mod_q=558
 8. key=52778   loglike=-412.1910   mod_q=2843
 9. key=13874   loglike=-473.4878   mod_q=558
10. key=46120   loglike=-479.0464   mod_q=2843

========== TOP 10 KEY GUESSES for a1 ==========
 1. key=   17   loglike=-183.2087   mod_q=17
 2. key= 3346   loglike=-219.0249   mod_q=17
 3. key=62224   loglike=-250.7990   mod_q=2302
 4. key=58895   loglike=-318.8102   mod_q=2302
 5. key=55566   loglike=-387.1286   mod_q=2302
 6. key= 6675   loglike=-388.6005   mod_q=17
 7. key=10004   loglike=-395.9701   mod_q=17
 8. key=13333   loglike=-434.1951   mod_q=17
 9. key=16662   loglike=-437.6814   mod_q=17
10. key=52237   loglike=-4